In [44]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns 

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, metrics, tree, svm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn.externals.six import StringIO
from sklearn.metrics import classification_report, confusion_matrix, jaccard_similarity_score, log_loss, f1_score, mean_absolute_error 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [67]:
# Import Dataframe
df = pd.read_csv('df_FeatureSelection.csv')

In [68]:
df.drop('SalePrice', axis = 1, inplace = True)
df.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,TotalBsmtSF,HeatingQC,LowQualFinSF,GrLivArea,FullBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,Latitude,Longitude,restaurants,stores,transport,school,nightlife,golf,park,grocery,service,recreational,cultural,leisure,SalePrice_log,age,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Bloomington Heights,Neighborhood_Bluestem,Neighborhood_Briardale,Neighborhood_Brookside,Neighborhood_Clear Creek,Neighborhood_College Creek,Neighborhood_Crawford,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Meadow Village,Neighborhood_Mitchell,Neighborhood_Northpark Villa,Neighborhood_Northridge,Neighborhood_Northridge Heights,Neighborhood_Northwest Ames,Neighborhood_Old Town,Neighborhood_Sawyer,Neighborhood_Sawyer West,Neighborhood_Somerset,Neighborhood_South & West of Iowa State University,Neighborhood_Stone Brook,Neighborhood_Timberland,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,GarageType_2Types,GarageType_BuiltIn,GarageType_None,GarageType_Separate,GarageFinish_0,GarageFinish_1,GarageFinish_2,GarageFinish_None,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,65.0,8450,7,5,2003,196.0,4,3,0,856,5,0,1710,2,3,4,8,0,1,2,548,3,0,61,0,0,0,0,0,0,2,42.023708,-93.647577,1,1,0,1,1,0,0,1,1,1,1,0,12.247694,5,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0

In [69]:
# Create X and y dataframes
X = df.iloc[:,1:]
X.drop('SalePrice_log', axis = 1, inplace = True)
y = df['SalePrice_log'].values

# Drop Neighborhood columns
X = X[X.columns.drop(list(X.filter(regex='Neighborhood')))]

# STANDARDIZE
X = preprocessing.StandardScaler().fit(X).transform(X)

# Split into test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#
fs_api =  ['restaurants', 'stores', 'transport', 'school', 'nightlife', 'golf', 'park', 
           'grocery', 'service', 'recreational', 'cultural', 'leisure']



In [71]:
y

array([12.24769432, 12.31716669, 12.54075757, ..., 11.83101549,
       11.92503512, 11.72803684])

Spot check algorithms using cross-validation

In [70]:
pipelines = []
pipelines.append(('LinearRegression', Pipeline([('LR',LinearRegression())])))
pipelines.append(('LassoRegression', Pipeline([('LASSO', Lasso())])))
pipelines.append(('ElasticNet', Pipeline([('EN', ElasticNet())])))
pipelines.append(('RandomForest', Pipeline([('RF', RandomForestRegressor())])))
pipelines.append(('SupportVector', Pipeline([('SVR', SVR())])))
pipelines.append(('GradientBoosting', Pipeline([('GBM', GradientBoostingRegressor())])))

kfold = KFold(n_splits=10)

results = []
names = []

for name, model in pipelines:
    
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LinearRegression: -2737525887.987487 (5018503305.658868)
LassoRegression: -0.318790 (0.017632)
ElasticNet: -0.318790 (0.017632)
RandomForest: -0.104729 (0.005803)
SupportVector: -0.129571 (0.016901)
GradientBoosting: -0.097654 (0.005803)


As can be seen from the neg_mean_absolute_error we observe a large error in the linear regression. This could be an effect of breaches to the assumptions underlying the regression model. Most likely highly correlated variables.

Lasso and Elastic net makes more sense than a linear regression as we have a normalization effect
A random forest approach makes a lot of sense since we have a lot of binary variables (0-1 features).


We see that support vectors, random forest and gradient boosting regression models perform better than the rest on the train set.

As Random Forest is a tree-based model, we have a problem with our many categorical variables. From a splitting algorithms POW, all binary features are independent, and as a result, continuous variables are assigned higher importance and chosen as the top tree splits. One could alleviate this issue by using a H2o framework rather than SkLearn since H2o does not require categorical features to be one-hot encoded. This potential improvement is considered out of the scope of this project.

## Model Tuning

#### Random Forest Tuning (Start 3:09 pm)

In [ ]:
parameters = {'n_estimators': range(10, 100, 10), 
              'max_depth': range(10, 40, 5),
              'max_features': ('auto', 'sqrt', 'log2')
             }

rf = RandomForestRegressor(criterion = 'mae')

gs_rf = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_rf.fit(X_train, y_train)

In [73]:
print(gs_rf.best_score_)
print(gs_rf.best_estimator_)

-0.10572993304701936
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=60, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)


#### Support Vector Tuning

In [ ]:
parameters = {'C': range(1, 10, 1), 
              'kernel': ('linear', 'sigmoid', 'rbf', 'poly')
             }

sv = SVR()
gs_sv = GridSearchCV(sv, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_sv.fit(X_train, y_train)

In [ ]:
print(gs_sv.best_score_)
print(gs_sv.best_estimator_)


#### Gradient Boosting Tuning

In [ ]:
parameters = {'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1), 
              'n_estimators': range(50, 200, 50),
              'loss': ('ls', 'lad', 'huber', 'quantile')
             }

gb = GradientBoostingRegressor()
gs_gb = GridSearchCV(gb, parameters, scoring = 'neg_mean_absolute_error', cv = kfold)
gs_gb.fit(X_train, y_train)

In [ ]:
print(gs_gb.best_score_)
print(gs_gb.best_estimator_)

# Print results
print("Best: %f using %s" % (gs_gb.best_score_, gs_gb.best_params_))
means = gs_gb.cv_results_['mean_test_score']
stds = gs_gb.cv_results_['std_test_score']
params = gs_gb.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## Test ensembles

In [ ]:
tpred_rf = gs_rf.best_estimator_.predict(X_test)
tpred_sv = gs_sv.best_estimator_.predict(X_test)
tpred_gb = gs_gb.best_estimator_.predict(X_test)

In [ ]:
print('rf mae:', mean_absolute_error(y_test, tpred_rf))
print('sv mae:', mean_absolute_error(y_test, tpred_sv))
print('gb mae:', mean_absolute_error(y_test, tpred_gb))
print('combined mae:', mean_absolute_error(y_test, (tpred_rf + tpred_sv + tpred_gb) / 3)